In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import torch
import torch.nn as nn
from torch.nn import Parameter
import sys
sys.path.append('.')

from sklearn.metrics import classification_report
from utils import time_since
import traceback

import time

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
from torch.autograd import Variable

import math, random
import argparse
from collections import deque
import cPickle as pickle

from fast_jtnn import *
import rdkit

In [2]:
label_col = 10 + 3 
target_unlabeled_percentage = 0.75
early_stop_thresh = 5

load_epoch = 0
alpha = 10 #0.1 * len_unlabelled / len_labelled

hidden_size=56 #450
batch_size=8
latent_size=28
depthT=20
depthG=3
y_size=2

lr=1e-3
clip_norm=50.0
beta=0.0
step_beta=0.002
max_beta=1.0
warmup=40 #40000

total_epoch=50
test_epoch=1
anneal_rate=0.9
anneal_iter=40 #40000
kl_anneal_iter=20 #2000

save_iter=50
print_iter=5

num_workers = 4
has_cuda = torch.cuda.is_available()

save_dir = 'data/tox21/model'
train_folder = 'fast_molvae/tox21_processed/train'
test_folder = 'fast_molvae/tox21_processed/test'
vocab_file = 'data/tox21/vocab.txt'

In [3]:
vocab = [x.strip("\r\n ") for x in open(vocab_file)]
vocab = Vocab(vocab)

In [4]:
weight = torch.tensor([1., 5.])

semiJTNNVAE = SemiJTNNVAEClassifier(vocab, hidden_size, latent_size, y_size, depthT, depthG, alpha=100, weight=weight)
semiNNVAE = NNVAE(1024, hidden_size, latent_size, y_size, alpha=10)

def setup(model):
    if has_cuda: model = model.cuda()

    for param in model.parameters():
        if param.dim() == 1:
            nn.init.constant_(param, 0)
        else:
            nn.init.xavier_normal_(param)

    # if load_epoch > 0:
    #     model.load_state_dict(torch.load(save_dir + "/model.iter-" + str(load_epoch)))
    # param_norm = lambda m: math.sqrt(sum([p.norm().item() ** 2 for p in m.parameters()]))
    # grad_norm = lambda m: math.sqrt(sum([p.grad.norm().item() ** 2 for p in m.parameters() if p.grad is not None]))

    print "Model #Params: %dK" % (sum([x.nelement() for x in model.parameters()]) / 1000,)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.ExponentialLR(optimizer, anneal_rate)
    
    return model, optimizer, scheduler

semiJTNNVAE, jt_optimizer, jt_scheduler = setup(semiJTNNVAE)
semiNNVAE, nn_optimizer, nn_scheduler = setup(semiNNVAE)

/home/tsa87/miniconda3/envs/python2/lib/python2.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Model #Params: 183K
Model #Params: 178K


In [5]:
# Doing this because 20% of labelled data got lost in test
tracker = IndexTracker(train_folder, label_idx=label_col, label_pct=(1-target_unlabeled_percentage)/0.8)

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [6]:
def train(model, optimizer, scheduler, use_moltree=True, patience=3):
    best_loss = np.inf
    no_improvement = 0
    
    beta = 0
    total_step = load_epoch
    
    start = time.time()
    for epoch in xrange(total_epoch):
        
        preds = np.array([])
        targets = np.array([])

        # Evaluation loop
        if epoch % test_epoch == 0:
            with torch.no_grad():
                model.eval()

                if use_moltree:
                    test_loader = MolTreeFolder(test_folder, vocab, batch_size=batch_size, feature_idx=1, label_idx=label_col, num_workers=num_workers)
                else:
                    test_loader = BaseFolder(test_folder, batch_size=batch_size, feature_idx=2, label_idx=label_col, num_workers=num_workers)
                    
                cum_loss = 0
                for (supervised_batch, unsupervised_batch) in test_loader:  
                    try:
                        if len(supervised_batch['labels']) == batch_size:
                            supervised_input = supervised_batch['data']
                            labels = supervised_batch['labels'].cuda() if torch.cuda.is_available() else supervised_batch['labels']
                                
                            loss, clsf_loss, clsf_acc, (pred, target) = model(supervised_input, labels, beta)
                            
                            cum_loss += loss

                            preds = np.append(preds, pred.cpu().detach().numpy())
                            targets = np.append(targets, target.cpu().detach().numpy())
                                            
                    except Exception as e:
                        traceback.print_exc()
                        continue

                print(classification_report(targets, preds)) 
                print 'time: %s' % time_since(start)
                print "[Test] Loss: %.3f, Clsf_loss: %.2f" % (loss, clsf_loss)
                
                if cum_loss < best_loss:
                    best_loss = cum_loss
                    no_improvement = 0
                else:
                    no_improvement += 1
                    if no_improvement > patience:
                        print "Ran out of patience, stop"
                        return 
                
                
        if use_moltree:
            train_loader = MolTreeFolder(train_folder, vocab, batch_size=batch_size, feature_idx=1, label_idx=label_col, num_workers=num_workers)
        else:
            train_loader = BaseFolder(train_folder, batch_size=batch_size, feature_idx=2, label_idx=label_col, num_workers=num_workers)
        
        cache = np.zeros(3)
        model.train()

        for (supervised_batch, unsupervised_batch) in train_loader: 
            try:
                if len(supervised_batch['labels']) == batch_size and len(unsupervised_batch['labels']) == batch_size:
                    model.zero_grad()

                    total_step += 1
                    labels = supervised_batch['labels'].cuda() if torch.cuda.is_available() else supervised_batch['labels']
                    
                    unsupervised_loss, _, _, _ = model(unsupervised_batch['data'], None, beta)
                    supervised_loss, clsf_loss, clsf_acc, (pred, target) = model(supervised_batch['data'], labels, beta)
            
                    loss = unsupervised_loss + supervised_loss
                    
                    cache += np.array([loss, clsf_loss, clsf_acc], dtype=np.float)
                    
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), clip_norm)
                    optimizer.step()
                    optimizer.zero_grad()

                    if total_step % save_iter == 0:
                        print "Saving model to " + save_dir + "/model.iter-" + "at step: " + str(total_step)
                        torch.save(model.state_dict(), save_dir + "/model.iter-" + str(total_step))

                    if total_step % anneal_iter == 0:
                        scheduler.step()
                        print "learning rate: %.6f" % scheduler.get_lr()[0]

                    if total_step % kl_anneal_iter == 0 and total_step >= warmup:
                        beta = min(max_beta, beta + step_beta)

                    if total_step % print_iter == 0:
                        cache /= print_iter
                        print "Epoch: %d | Iter: %d" % (epoch, total_step)
                        print "[Test] Loss: %.3f, Clsf_loss: %.2f, Clsf_acc: %.2f" % (cache[0], cache[1], cache[2])
                        sys.stdout.flush()
                        cache *= 0

            except Exception as e:
                traceback.print_exc()
                continue

        scheduler.step()

In [ ]:
train(semiJTNNVAE, jt_optimizer, jt_scheduler, True)

fast_molvae/tox21_processed/test/tensors-10.pkl opened
fast_molvae/tox21_processed/test/tensors-9.pkl opened


/home/tsa87/miniconda3/envs/python2/lib/python2.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/tsa87/miniconda3/envs/python2/lib/python2.7/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/tsa87/miniconda3/envs/python2/lib/python2.7/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/home/tsa87/miniconda3/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, wa

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       166
         1.0       0.14      1.00      0.24        26

   micro avg       0.14      0.14      0.14       192
   macro avg       0.07      0.50      0.12       192
weighted avg       0.02      0.14      0.03       192

time: 1 m 52 s
[Test] Loss: 229.864, Clsf_loss: 109.80
fast_molvae/tox21_processed/train/tensors-6.pkl opened
Epoch: 0 | Iter: 5
[Test] Loss: 321.322, Clsf_loss: 118.62, Clsf_acc: 0.17


Traceback (most recent call last):
  File "<ipython-input-6-e1dd9347bfd5>", line 73, in train
    supervised_loss, clsf_loss, clsf_acc, (pred, target) = model(supervised_batch['data'], labels, beta)
  File "/home/tsa87/miniconda3/envs/python2/lib/python2.7/site-packages/torch/nn/modules/module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "fast_jtnn/jtnn_vae.py", line 338, in forward
    self._compute_partial_loss(y_batch, x_batch, x_tree_vecs, x_tree_mess, x_mol_vecs, x_jtmpn_holder, beta)
  File "fast_jtnn/jtnn_vae.py", line 276, in _compute_partial_loss
    word_loss, topo_loss, word_acc, topo_acc = self.decoder(x_batch, z_tree_vecs)
  File "/home/tsa87/miniconda3/envs/python2/lib/python2.7/site-packages/torch/nn/modules/module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "fast_jtnn/jtnn_dec.py", line 108, in forward
    cur_h_nei = torch.stack(cur_h_nei, dim=0).view(-1,MAX_NB,self.hidden_size)
RuntimeError: shape '[-1

Epoch: 0 | Iter: 10
[Test] Loss: 262.406, Clsf_loss: 81.64, Clsf_acc: 0.07
Epoch: 0 | Iter: 15
[Test] Loss: 341.179, Clsf_loss: 123.59, Clsf_acc: 0.20


...

Epoch: 4 | Iter: 905
[Test] Loss: 296.330, Clsf_loss: 148.59, Clsf_acc: 1.45
fast_molvae/tox21_processed/train/tensors-1.pkl opened
Epoch: 4 | Iter: 910
[Test] Loss: 202.981, Clsf_loss: 93.22, Clsf_acc: 0.62
Epoch: 4 | Iter: 915
[Test] Loss: 150.705, Clsf_loss: 67.34, Clsf_acc: 0.68
learning rate: 0.000089
Epoch: 4 | Iter: 920
[Test] Loss: 175.151, Clsf_loss: 88.65, Clsf_acc: 0.78
Epoch: 4 | Iter: 925
[Test] Loss: 148.533, Clsf_loss: 69.55, Clsf_acc: 0.80
Epoch: 4 | Iter: 930
[Test] Loss: 163.265, Clsf_loss: 81.22, Clsf_acc: 0.72
Epoch: 4 | Iter: 935
[Test] Loss: 184.994, Clsf_loss: 95.03, Clsf_acc: 0.82
fast_molvae/tox21_processed/train/tensors-7.pkl opened
Epoch: 4 | Iter: 940
[Test] Loss: 175.715, Clsf_loss: 87.40, Clsf_acc: 0.82
Epoch: 4 | Iter: 945
[Test] Loss: 165.185, Clsf_loss: 83.78, Clsf_acc: 0.75


Traceback (most recent call last):
  File "<ipython-input-6-e1dd9347bfd5>", line 73, in train
    supervised_loss, clsf_loss, clsf_acc, (pred, target) = model(supervised_batch['data'], labels, beta)
  File "/home/tsa87/miniconda3/envs/python2/lib/python2.7/site-packages/torch/nn/modules/module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "fast_jtnn/jtnn_vae.py", line 338, in forward
    self._compute_partial_loss(y_batch, x_batch, x_tree_vecs, x_tree_mess, x_mol_vecs, x_jtmpn_holder, beta)
  File "fast_jtnn/jtnn_vae.py", line 276, in _compute_partial_loss
    word_loss, topo_loss, word_acc, topo_acc = self.decoder(x_batch, z_tree_vecs)
  File "/home/tsa87/miniconda3/envs/python2/lib/python2.7/site-packages/torch/nn/modules/module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "fast_jtnn/jtnn_dec.py", line 108, in forward
    cur_h_nei = torch.stack(cur_h_nei, dim=0).view(-1,MAX_NB,self.hidden_size)
RuntimeError: shape '[-1

Saving model to data/tox21/model/model.iter-at step: 950


Traceback (most recent call last):
  File "<ipython-input-6-e1dd9347bfd5>", line 86, in train
    torch.save(model.state_dict(), save_dir + "/model.iter-" + str(total_step))
  File "/home/tsa87/miniconda3/envs/python2/lib/python2.7/site-packages/torch/serialization.py", line 260, in save
    return _with_file_like(f, "wb", lambda f: _save(obj, f, pickle_module, pickle_protocol))
  File "/home/tsa87/miniconda3/envs/python2/lib/python2.7/site-packages/torch/serialization.py", line 183, in _with_file_like
    f = open(f, mode)
IOError: [Errno 2] No such file or directory: 'data/tox21/model/model.iter-950'


Epoch: 4 | Iter: 955
[Test] Loss: 349.779, Clsf_loss: 188.37, Clsf_acc: 1.50
learning rate: 0.000080
Epoch: 4 | Iter: 960
[Test] Loss: 178.275, Clsf_loss: 86.84, Clsf_acc: 0.85
fast_molvae/tox21_processed/train/tensors-3.pkl opened


In [ ]:
train(semiNNVAE, nn_optimizer, nn_scheduler, False)